# 제주도 버스승차 인원 예측
- ## data 415423 rows × 49 columns
### Columns
- id : 해당 데이터에서의 고유한 id
- data : 날짜
- bus_route_id : 노선 ID
- in_out : 시내버스, 시외버스 구분
- station_code : 해당 승하차 정류소의 id
- station_name : 해당 승하차 정류소의 이름
- latitude : 해당 버스정류장의 위도
- longitude : 해당 버스정류장의 경도
- X-Y_ride : X:00:00 ~ x:59:59까지 승차한 인원 수
- X-Y_takeoff : X:00:00 ~ x:59:59까지 하차한 인원 수
- 18-20_ride : 18:00:00부터 19:59:59까지 승차한 인원 수
- 지번주소 : EDA 및 제주 날씨 merge용 column 
- si :EDA 및 제주 날씨 merge용 column 
- city : EDA 및 제주 날씨 merge용 column  
- dong : EDA 및 제주 날씨 merge용 column  
- road_addr : EDA 및 제주 날씨 merge용 column  
- weather_addr : EDA 및 제주 날씨 merge용 column 
- temperature : 온도
- precipitation : 강수량
- bus_interval : 노선 배차 간격
- date_day : 일 (1일 ~ 30일)
- date_dayofweek : 요일 (0 ~ 6)
- date_dayofname : 요일 (월 ~ 일)
- weekday : 평일 여부
- weekend : 주말 여부
- holiday : 공휴일 여부
- typhoon : 태풍이 온날
- in_ : 시내 버스 구분
- out : 시외 버스 구분
- ridexx : ride(탑승)의 2시간 간격 (ex 6시부터 ~7시 까지), 예측하는 y값이 18시 ~ 20시 까지이므로 생성함
- offxx : off(하차)의 2시간 간격 (ex 6시부터 ~7시 까지), 예측하는 y값이 18시 ~ 20시 까지이므로 생성함
- ride_sum : 승차인원의 합계
- off_sum : 하차인원의 합계
- bus_route_id_sum : 정류소, 일별 운행한 버스노선의 갯수
- bus_route_id_all_sum : 정류소별 9월 전체 운행한 버스노선의 갯수



# 이슈
    - zero_df로 뺀것은 전체 데이터에서 1개만 있는것들 입니다.
    - 정류장별로 듀플리케이트한것과 안한것과 갯수가 같은것 (토론떄는 zero_df 로 빼자고한것, 약 3700개)은 
    - 따로 예측은 하지않고 train 모델에만 넣는것으로 하였습니다. (사유 : 박사님 피드백)

### 0. 공통

In [4]:
# 추가되는 패키지는 여기에서 import 해주세요
import warnings
warnings.filterwarnings('ignore')
import missingno
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import pickle
from sklearn.externals import joblib
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from itertools import count
import plotly_express as px
from tqdm.notebook import tqdm
import geopy.distance

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 500)
# matplotlib 설정
mpl.use('Agg')
mpl.rcParams['axes.unicode_minus'] = False
%matplotlib inline
# seaborn 설정
sns.set()
sns.set_style("whitegrid")
sns.set_color_codes()
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

In [5]:
pd.set_option('max_columns', 700)
pd.set_option('max_rows', 500)

### 01. 데이터 불러오기

In [13]:
#데이터 불러오기
# 415423 rows × 49 columns
raw_data = pd.read_csv('raw_train.csv')
raw_data.tail(2)

,id,date,bus_route_id,in_out,station_code,station_name,latitude,longitude,ride6,ride7,ride8,ride9,ride10,ride11,off6,off7,off8,off9,off10,off11,ride18,지번주소,si,city,dong,road_addr,weather_addr,temperature,precipitation,bus_interval,date_day,date_dayofweek,date_dayofname,weekday,weekend,holiday,typhoon,in_,out,ride67,ride89,ride1011,off67,off89,off1011,ride_sum,off_sum,bus_route_id_sum,bus_route_id_all_sum
415421,415421,2019-09-30,32820000,시내,3291,애월환승정류장(애월리),33.46483,126.31870,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,제주특별자치도 제주시 애월읍 애월리 1534-2,제주시,애월읍,애월리,애월로,제주,23.6,22.6,8.0,30,0,Monday,1.0,0.0,0,0,1,0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,5,120
415422,415422,2019-09-30,32820000,시내,6115100,서귀포시외버스터미널,33.24873,126.50799,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,제주특별자치도 서귀포시 법환동 843,서귀포시,서귀포시,법환동,일주동로,서귀포,24.3,6.3,8.0,30,0,Monday,1.0,0.0,0,0,1,0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4.0,15,440


### 02. raw_data를 검증하기

In [21]:
# raw_data에서 1회만 운행한 정류장은 zero_df로 빼는 함수
# raw_train = 1회 운행한 정류장이 제외된 train_data
# zero_df = 1회 운행한 정류장의 간략한 정보만 가지고 있는 df(station_code, ride18)
# raw_zero = 1회 운행한 정류장의 df
def make_zero_raw_train(raw_data):
    table = pd.pivot_table(index='station_code', data=raw_data, values='id', aggfunc=len).reset_index()
    table = table[table['id'] == 1]
    raw_train = raw_data.copy()
    raw_zero =pd.DataFrame(columns=raw_data.columns)
    total = tqdm(table['station_code'].unique())
    for i in total:
        raw_train = raw_train[raw_train['station_code'] != i]
        raw_zero = raw_zero.append(raw_data[raw_data['station_code'] == i])
    # zero 모델의 df 생성
    a = raw_zero[['station_code','ride18']]
    zero_df = pd.merge(table, a, how = 'inner')
    return raw_train, zero_df, raw_zero

In [22]:
raw_train, zero_df, raw_zero = make_zero_raw_train(raw_data)

In [23]:
# zero_df를 제외한 raw_train에서 train, test 데이터를 나누는 함수 (중복제거 사용)
def make_train_test(dataframe, category, test_size, random_state):
    # 필수 train 데이터를 만들고
    nec_train = dataframe.drop_duplicates(subset=category)
    # 데이터프레임에서 필수 train을 제외시킨 train_test 셋을 만들고
    train_test = dataframe.drop(dataframe['id'][nec_train['id']])
    X, y = train_test_split(
        train_test, test_size=test_size, random_state=random_state)
    train = pd.concat([nec_train, X])
    return train, y

In [24]:
# 테스트 사이즈가 0.3 정도되야 실제 비율이 7.5:2.5 정도 됩니다.(중복제거된 데이터를 train에 붙이기 때문)
# 만일 카테고리 데이터가 추가된다면 여기에 추가하셔야 합니다.
# 랜덤 스테이트가 바뀌면 테스트 셋이 바뀌니 그냥 진행해주세요
category = ['station_code','bus_route_id', 'in_', 'out', 'weekend', 'weekday', 'holiday', 'typhoon']
df_train, df_test = make_train_test(raw_train, category, 0.3, 13)

In [25]:
# test와 train의 비율 및 정류소의 갯수 (둘다 동일하게 들어갔는지.)
print('train의 비율:', len(df_train) / len(raw_train))
print('test의 비율:', len(df_test) / len(raw_train))
print('전체의 정류소 갯수:', len(raw_train['station_code'].unique()))
print('전체의 노선 갯수:', len(raw_train['bus_route_id'].unique()))
print('train의 정류소 갯수:', len(df_train['station_code'].unique()))
print('test의 정류소 갯수:', len(df_test['station_code'].unique()))
print('train의 노선 갯수:', len(df_train['bus_route_id'].unique()))
print('test의 노선 갯수:', len(df_test['bus_route_id'].unique()))

train의 비율: 0.7682769447092928
test의 비율: 0.2317230552907072
전체의 정류소 갯수: 3472
전체의 노선 갯수: 613
train의 정류소 갯수: 3472
test의 정류소 갯수: 3197
train의 노선 갯수: 613
test의 노선 갯수: 574


In [26]:
# 데이터 스케일링
# 실수형 데이터가 추가되면 아래 values에 추가하면 됩니다.
values = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11', 'off6', 'off7', 'off8',
          'off9', 'off10', 'off11', 'ride18', 'temperature', 'precipitation', 'bus_interval',
          'ride67', 'ride89', 'ride1011', 'off67', 'off89', 'off1011', 'ride_sum', 'off_sum',
          'bus_route_id_sum', 'bus_route_id_all_sum']

scale_values = [f'scale_{value}' for value in values]

df_train[scale_values] = df_train[values] # train_data
df_test[scale_values] = df_test[values] # test_data
raw_zero[scale_values] = raw_zero[values] # zero_df

X = df_train[scale_values]
Y = df_test[scale_values]
Z = raw_zero[scale_values]

scaler = StandardScaler()
scaler.fit(X)

X_scaler = scaler.transform(X)
Y_scaler = scaler.transform(Y)
Z_scaler = scaler.transform(Z)

df_train[scale_values] = X_scaler
df_test[scale_values] = Y_scaler
raw_zero[scale_values] = Z_scaler

In [27]:
# split 데이터 검증용
# train + validation data를 받아서 임의의 정류장 갯수를 가져옴
def split(df_train, num, seed):
    
    test_frame = pd.DataFrame(columns=df_train.columns)
    np.random.seed(seed)

    for i in np.random.choice(df_train['station_code'].unique(), num, replace=False):
        df1 = df_train[df_train['station_code'] == i]
        test_frame = pd.concat([test_frame, df1])
    return test_frame


# 데이터 검증용 함수
# df_train(or test_frame)를 train_df(학습)와 validation_df(검주ㅡㅇ)를 만듬
# train에는 중복제거된 데이터가 포함되어있음
# train과 validation을 나누는 과정에서 random_state를 다르게 주면 데이터가 바뀜(Kfold와 비슷한 효과)
def make_train_validation(dataframe, cate, test_size, seed):
    train_df = pd.DataFrame(columns=dataframe.columns)
    validation_df = pd.DataFrame(columns=dataframe.columns)
    
    for i in dataframe['station_code'].unique():
        df1 = dataframe[dataframe['station_code'] == i]
        # 필수 train 데이터를 만들고
        nec_train = df1.drop_duplicates(subset=cate)
        # 필수 train 데이터를 제외한 train_validation을 만듬
        train_validation = df1.drop(df1['id'][nec_train['id']])
        # 만약 필수 train데이터와 train_validation의 크기가 같다(모두 고윳값이다)면
        if len(nec_train) == len(df1):
            #그냥 모두 train_df에 넣음,(validation에는 넣지않음, 박사님 피드백)
            train_df = pd.concat([train_df, nec_train])

        # 만약 train_validation의 갯수가 1이하면, train_validation은 바로 validation이 됨       
        elif len(train_validation) <= 1:
            # train_df와
            train_df = pd.concat([train_df, nec_train])
            # validation_df를 생성함
            validation_df = pd.concat([validation_df, train_validation])
        # 그 외에는 필수 train + train , validation 으로 나눠줌
        else:
            X, y = train_test_split(
                train_validation, test_size=test_size, random_state=seed)
            train_a = pd.concat([nec_train, X])
            train_df = pd.concat([train_df, train_a])
            validation_df = pd.concat([validation_df, y])
    return train_df, validation_df

# 검증 모델
# 만일 validation_df의 station_code가 없다면 학습만하고 예측을 하지 않음
def ols_validation(train_df, validation_df, var, cate):
    
    total = tqdm(train_df['station_code'].unique())
    columns = train_df.columns
    df_tr = pd.DataFrame(columns=columns)
    df_te = pd.DataFrame(columns=columns)
    df_tr['yhat'] = 999
    df_te['yhat'] = 999
    cate_c = [f"C({name})" for name in cate]
    y = ['scale_ride18']
    for i in total:
        train_ols = train_df[train_df['station_code'] == i]
        validation_ols = validation_df[validation_df['station_code'] == i]
    
        if len(validation_ols) ==0:
            model = sm.OLS.from_formula(
            'scale_ride18  ~ ' + '+'.join(var)
            + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])

        else :
            model = sm.OLS.from_formula(
            'scale_ride18  ~ ' + '+'.join(var)
            + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])
            
            validation_ols_df = validation_ols[var+cate]  # 테스트 모델
            validation_ols['yhat'] = result.predict(validation_ols_df)
            df_te = pd.concat([df_te, validation_ols])
    return df_tr, df_te

# R스퀘어 구하기
# 앞에서 train과 validation 분리한 seed를 넣어 어떤 seed가 어떤 결정계수가 나왔는지 확인한다.
# DataFrame형태로 반환
def get_rsquared(df_tr, df_te, seed):

    df_tr['residual'] = df_tr['scale_ride18'] - df_tr['yhat']
    df_tr['explained'] = df_tr['yhat'] - np.mean(df_tr['yhat'])
    df_tr['total'] = df_tr['scale_ride18'] - np.mean(df_tr['scale_ride18'])

    df_te['residual'] = df_te['scale_ride18'] - df_te['yhat']
    df_te['explained'] = df_te['yhat'] - np.mean(df_te['yhat'])
    df_te['total'] = df_te['scale_ride18'] - np.mean(df_te['scale_ride18'])

    train_ess = np.sum((df_tr['explained'] ** 2))
    train_rss = np.sum((df_tr['residual'] ** 2))
    train_tss = np.sum((df_tr['total'] ** 2))
    test_ess = np.sum((df_te['explained'] ** 2))
    test_rss = np.sum((df_te['residual'] ** 2))
    test_tss = np.sum((df_te['total'] ** 2))

    rsquared = {'seed': [f'{seed}'],
             'train_rsquared_1': [1-train_rss/train_tss],
             'train_rsquared_2': [train_ess/train_tss],
             'validation_rsquared_1': [1-test_rss/test_tss],
             'validation_rsquared_2': [test_ess/test_tss],
             'train_ESS' : [round(train_ess)],
             'train_RSS' : [round(train_rss)],
             'train_TSS' : [round(train_tss)],
             'validation_ESS' : [round(test_ess)],
             'validation_RSS' : [round(test_rss)],
             'validation_TSS' : [round(test_tss)],
             'train_RMSE' : [np.sqrt(((df_tr['scale_ride18'] - df_tr['yhat']) ** 2).mean())],
             'validation_RMSE' : [np.sqrt(((df_te['scale_ride18'] - df_te['yhat']) ** 2).mean())],
               }
    print(f'seed : {seed} 완료')
    return pd.DataFrame(rsquared)

In [28]:
# 한번에 여러번 해보기
# 위에서 만든 make_train_validation -> ols_validation -> get_rsquared의 순서를 거침
# 시드를 리스트로 받아서
# dataframe = (train과 validation으로 나누기 전의 데이터)
# seeds = 리스트형태의 seed 목록 (갯수만큼 ols를 검증함)
# test_size = validation_size, 왠만하면 0.2로 고정해주세요
# 리턴되는 rsquared_df 데이터 프레임에 train, validation의 결정계수, ess, rss, tss, RMSE를 seed 별로 데이터프레임으로 만들어줍니다.


def validations(dataframe, seeds, test_size):
    rsquared_df = pd.DataFrame()
    for seed in seeds:
        train_df, validation_df = make_train_validation(dataframe, cate, test_size, seed)
        df_tr, df_te = ols_validation(train_df, validation_df, var, cate)
        rsquared = get_rsquared(df_tr, df_te, seed)
        rsquared_df = pd.concat([rsquared_df, rsquared])
    return rsquared_df

In [32]:
# 여기서 변수 변경해가면서 검증해보세요
var_total = ['scale_ride6','scale_ride7','scale_ride8','scale_ride9', 'scale_ride10','scale_ride11',
       'scale_off6','scale_off7','scale_off8','scale_off9','scale_off10','scale_off11',
       'scale_temperature','scale_precipitation','scale_bus_interval',
       'scale_ride67','scale_ride89','scale_ride1011','scale_off67','scale_off89', 'scale_off1011',
        'scale_ride_sum','scale_off_sum','scale_bus_route_id_sum','scale_bus_route_id_all_sum']

# 'scale_ride67','scale_ride89','scale_ride1011','scale_off67','scale_off89', 'scale_off1011' 
# 는 2시간 더한 컬럼입니다.

# 검증에 사용할 실수 변수를 넣으세요
var = ['scale_ride6','scale_ride7','scale_ride8','scale_ride9', 'scale_ride10','scale_ride11',
       'scale_off6','scale_off7','scale_off8','scale_off9','scale_off10','scale_off11',
       'scale_temperature','scale_precipitation','scale_bus_interval',
        'scale_ride_sum','scale_off_sum','scale_bus_route_id_sum','scale_bus_route_id_all_sum']

# 검증에 사용할 카테고리 변수를 넣으세요
cate = ['bus_route_id', 'in_', 'out','weekend', 'weekday', 'holiday', 'typhoon']

seeds = [10, 20, 30, 40, 50] # 랜덤시드를 주어서 Kfold한 효과를 가져옴
test_size = 0.3 #변경해도 상관은 없는데 그냥 0.3로 하는게.. 그래야 7.5:2.5 정도 나옵니다.

dataframe = split(df_train, 100, 50) # 정류장 100개만 해봅시다
rsquared_df = validations(dataframe, seeds, test_size)
rsquared_df


seed : 10 완료



seed : 20 완료



seed : 30 완료



seed : 40 완료



seed : 50 완료


,seed,train_rsquared_1,train_rsquared_2,validation_rsquared_1,validation_rsquared_2,train_ESS,train_RSS,train_TSS,validation_ESS,validation_RSS,validation_TSS,train_RMSE,validation_RMSE
0,10,0.630423,0.633658,0.238756,1.007502,1058.0,617.0,1670.0,533.0,403.0,529.0,0.306275,0.476423
0,20,0.635346,0.638139,0.438902,0.754417,1092.0,624.0,1712.0,369.0,274.0,489.0,0.307968,0.393325
0,30,0.651752,0.654912,0.199047,1.161767,1145.0,609.0,1748.0,524.0,361.0,451.0,0.304162,0.451408
0,40,0.630531,0.632509,0.224160,1.053404,1080.0,631.0,1708.0,518.0,381.0,491.0,0.309651,0.463541
0,50,0.628227,0.630780,0.473404,0.820521,1104.0,651.0,1751.0,369.0,237.0,450.0,0.314480,0.365336


### 03. 전체 데이터로 최종 확인
 - 함수 내에서 zero_df를 따로 생성하지 않습니다. 
 - 그러니 앞에 zero_df 생성하는 make_zero_raw_train 함수가 실행되어있어야 합니다.
 - 또한 zero_df의 변수명도 바뀌면 오류납니다.

In [33]:
# 나뉘어진 트레인과 테스트를 입력받아서, 정류장별 학습 및 예측 하면됨
# 최종 테스트

def ols_final(df_train, df_test, var, cate):

    total = tqdm(df_test['station_code'].unique())
    columns = df_train.columns
    df_tr = pd.DataFrame(columns=columns)
    df_te = pd.DataFrame(columns=columns)
    df_tr['yhat'] = 999
    df_te['yhat'] = 999
    cate_c = [f"C({name})" for name in cate]
    y = ['scale_ride18']
    for i in total:
        train_ols = df_train[df_train['station_code'] == i]
        test_ols = df_test[df_test['station_code'] == i]

        if i in zero_df['station_code'].unique():
            
            test_ols['yhat'] = zero_df[zero_df['station_code'] == i]['scale_ride18']
            df_tr = pd.concat([df_tr, test_ols])
            df_te = pd.concat([df_te, test_ols])

        elif len(test_ols) == 0:
            model = sm.OLS.from_formula(
                'scale_ride18  ~ ' + '+'.join(var)
                + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])

        else:
            model = sm.OLS.from_formula(
                'scale_ride18  ~ ' + '+'.join(var)
                + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])

            test_ols_df = test_ols[var+cate]  # 테스트 모델
            test_ols['yhat'] = result.predict(test_ols_df)
            df_te = pd.concat([df_te, test_ols])
    return df_tr, df_te

In [34]:
# 일단 전체를 돌릴순 없으니까 임의로 몇개만 빼서 해보자
# cate변수는 위에 선언되어있습니다. (카테고리 데이터)
dataframe = split(df_train, 100, 50)
x, y = make_train_validation(dataframe, cate, test_size, 10)

In [35]:
df_tr2, df_te2 = ols_final(x, y,var,cate)

In [36]:
# 결정계수 확인, 최종 확인에서는 seed값을 아무거나 주어도 상관없어요. 데이터프레임 만드려고 넣는거라..
get_rsquared(df_tr2, df_te2,0)

seed : 0 완료


,seed,train_rsquared_1,train_rsquared_2,validation_rsquared_1,validation_rsquared_2,train_ESS,train_RSS,train_TSS,validation_ESS,validation_RSS,validation_TSS,train_RMSE,validation_RMSE
0,0,0.630357,0.63359,0.238756,1.007502,1058.0,617.0,1670.0,533.0,403.0,529.0,0.306531,0.476423


In [37]:
# 전체 돌려보자 (zero는 제외)
df_tr2, df_te2 = ols_final(df_train, df_test, var, cate)

In [38]:
get_rsquared(df_tr2, df_te2,0)

seed : 0 완료


,seed,train_rsquared_1,train_rsquared_2,validation_rsquared_1,validation_rsquared_2,train_ESS,train_RSS,train_TSS,validation_ESS,validation_RSS,validation_TSS,train_RMSE,validation_RMSE
0,0,0.794942,0.794958,0.447833,1.030059,253487.0,65386.0,318868.0,129706.0,69529.0,125921.0,0.453748,0.849966


# zero model 확인
 - 우리가 가지고있는 Data로 말고, 실제 외부데이터가 들어왔다는 가정일때
 - 실제 test 모델 검증중엔 안해도 됩니다. 그냥 데이콘에 제출할수 있을까해서 만든거에요
 - 얘는 yhat을 scale이 안된값을 출력해야해서 합니다..포뮬라식에서 y가 ride_18 입니다

In [ ]:
# 실제 train 데이터를 입력받아서, zero_df를 만들고 실행합니다.
def ols_final_dacon(df_train, df_test, var, cate):

    columns = df_train.columns
    df_tr = pd.DataFrame(columns=columns)
    df_te = pd.DataFrame(columns=columns)
    df_tr['yhat'] = 999
    df_te['yhat'] = 999
    cate_c = [f"C({name})" for name in cate]
    y = ['ride18']
    
    # 입력받은 train 데이터에서 zero_model 생성
    # 입력받은 train은 우리가 임의로 나눈 데이터가 아니여도 zero_model이 있을수 있으므로 생성
    # 위에서 raw_data에서 뺸 zero_model은 검증을 위해 제외한것, 그것을 전체데이터를 검증하는 곳에 넣었으니
    # 여기서입력받는 df_train에는 위에서 제외시킨 zero_model이 포함되어야 함
    table = pd.pivot_table(index='station_code', data=df_train, values='id', aggfunc=len).reset_index()
    table = table[table['id'] == 1]
    raw_train = df_train.copy()
    raw_zero =pd.DataFrame(columns=df_train.columns)
    
    for i in table['station_code'].unique():
        raw_train = raw_train[raw_train['station_code'] != i]
        raw_zero = raw_zero.append(raw_data[raw_data['station_code'] == i])
    # zero 모델의 df 생성
    a = raw_zero[['station_code','ride18']]
    zero_df = pd.merge(table, a, how = 'inner')
    
    total = tqdm(df_test['station_code'].unique())
    for i in total:
        train_ols = df_train[df_train['station_code'] == i]
        test_ols = df_test[df_test['station_code'] == i]

        if i in zero_df['station_code'].unique():
            
            test_ols['yhat'] = zero_df[zero_df['station_code'] == i]['ride18']
            df_tr = pd.concat([df_tr, test_ols])
            df_te = pd.concat([df_te, test_ols])

        elif len(test_ols) == 0:
            model = sm.OLS.from_formula(
                'ride18  ~ ' + '+'.join(var)
                + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])

        else:
            model = sm.OLS.from_formula(
                'ride18  ~ ' + '+'.join(var)
                + '+'.join('+') + '+'.join(cate_c), data=train_ols)
            # 학습
            result = model.fit()
            # 결과
            train_ols['yhat'] = result.predict(train_ols)
            # 학습 저장
            df_tr = pd.concat([df_tr, train_ols])

            test_ols_df = test_ols[var+cate]  # 테스트 모델
            test_ols['yhat'] = result.predict(test_ols_df)
            df_te = pd.concat([df_te, test_ols])
    return df_tr, df_te

In [ ]:
# zero model만 돌리기

var = ['ride6', 'ride7', 'ride8', 'ride9', 'ride10', 'ride11',
       'off6', 'off7', 'off8', 'off9', 'off10', 'off11',
       'temperature', 'precipitation', 'bus_interval',
       'ride_sum', 'off_sum']
cate = ['bus_route_id', 'in_', 'out','weekend', 'weekday', 'holiday', 'typhoon']
df_tr2, df_te2 = ols_final_dacon(raw_zero, raw_zero,var,cate)

In [ ]:
# zero model을 포함한것 돌리기 (실제 외부데이터가 들어왔다는 가정으로)
# df_train = train data
# df_test = test data (zero_df 제외)
# zero_df = 1회만 운행된 정류소(예측모델 생성 완전히 불가)
# tm = df_train + zero_df (zero model 확인용으로 사용, 실제 모델은 df_train, df_test를 사용하면 됨)외부 데이터인것으로 가정
# im = df_test + zero_df (zero model 확인용으로 사용, 실제 모델은 df_train, df_test를 사용하면 됨)외부 데이터인것으로 가정

tm = pd.concat([raw_zero, df_train])
im = pd.concat([raw_zero, df_test])

df_tr2, df_te2 = ols_final_dacon(tm, im, var, cate)

In [ ]:
get_rsquared(df_tr2, df_te2,0)